In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from PIL import Image, UnidentifiedImageError
import os
from torch.utils.data import Dataset, DataLoader
import requests
# import albumentations as A
from torchvision import transforms
from tqdm.notebook import tqdm_notebook as tqdm
from io import BytesIO

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [42]:
data = pd.read_excel('/kaggle/input/adobe-behaviour-simulation/behaviour_simulation_train.xlsx')

In [43]:
from sklearn.model_selection import GroupKFold

In [44]:
data.columns

Index(['id', 'date', 'likes', 'content', 'username', 'media',
       'inferred company'],
      dtype='object')

In [45]:
data['date'] = pd.to_datetime(data['date'])
data['hour'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek  # 0=Monday, 6=Sunday


In [61]:
import re

# Define functions to count hashtags and mentions
def count_hashtags(text):
    return len(re.findall(r'#\w+', text))

def count_mentions(text):
    return len(re.findall(r'@\w+', text))

# Apply these functions to the tweet content
data['num_hashtags'] = data['content'].apply(count_hashtags)
data['num_mentions'] = data['content'].apply(count_mentions)
data['tweet_length'] = data['content'].apply(len)

In [62]:
gkf = GroupKFold()
for i, j in gkf.split(data, groups = data['inferred company']):
    X_train_1 = data.iloc[i]
    X_train_2 = data.iloc[j]
    break


In [63]:
X_train_1.shape

(240000, 12)

In [64]:
X_train_1.shape

(240000, 12)

In [65]:
Y_train_1 = X_train_1['likes']
X_train_1 = X_train_1.drop(['likes'], axis = 1)

In [66]:
Y_train_2 = X_train_2['likes']
X_train_2 = X_train_2.drop(['likes'], axis = 1)

In [67]:
for col in ['inferred company', 'username', 'id', 'media', 'date', 'content']:
    X_train_1 = X_train_1.drop([col], axis = 1)
    X_train_2 = X_train_2.drop([col], axis = 1)

In [68]:
# for col in X_train_1.columns:
#     if(type(X_train_1[col]) not in ['int', 'float', 'bool']):
#         X_train_1 = X_train_1.drop([col], axis = 1)
#         X_train_2 = X_train_2.drop([col], axis = 1)

In [69]:
X_train_1

,hour,day_of_week,num_hashtags,num_mentions,tweet_length
0,0,5,1,0,181
2,19,1,0,0,104
3,11,3,0,0,140
5,16,6,0,0,181
7,22,1,4,0,241
...,...,...,...,...,...
299993,1,3,0,0,68
299994,12,1,2,0,189
299996,11,4,1,0,151
299997,20,2,0,0,70


In [34]:
from lightgbm import LGBMRegressor

In [35]:
lgb = LGBMRegressor()

In [70]:
lgb.fit(X_train_1,Y_train_1)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019669 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 4
[LightGBM] [Info] Start training from score 873.058529


LGBMRegressor()

In [71]:
y_pred = lgb.predict(X_train_2)

In [74]:
from sklearn.metrics import mean_squared_error

In [75]:
mse = mean_squared_error(Y_train_2,y_pred)

In [76]:
np.sqrt(mse)

3160.2897284664346

In [81]:
pd.DataFrame(y_pred).describe()

,0
count,60000.000000
mean,995.060752
std,600.407260
min,-6556.697517
25%,745.697979
50%,871.738123
75%,1222.960688
max,28052.421633
